In [ ]:
import gzip
import os

import cudf
from cuml import KNN as cumlKNN
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KDTree as skKNN

# Helper Functions

In [ ]:
def load_data(nrows, ncols, cached="data/mortgage.npy.gz", source="mortgage"):
    """Load mortgage dataset if available, otherwise use random data."""
    if os.path.exists(cached) and source == "mortgage":
        print("Using mortgage data.")
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0, X.shape[0] - 1, nrows), :ncols]
    else:
        print("Using random data.")
        X = np.random.random((nrows, ncols)).astype("float32")
    df = pd.DataFrame({"fea%d" %i: X[:, i] for i in range(X.shape[1])}).fillna(0)
    return df

In [ ]:
def array_equal(a, b, threshold=1e-2, with_sign=True, metric="mse"):
    """Check the equality of two arrays with a given threshold and metric."""
    a = to_nparray(a)
    b = to_nparray(b)
    if not with_sign:
        a, b = np.abs(a), np.abs(b)
    if metric == "mse":
        error = mean_squared_error(a, b)
    else:
        error = np.sum(a != b) / (a.shape[0] * a.shape[1])
    res = error < threshold
    return res

def to_nparray(x):
    """Convert a given array into a numpy array."""
    if isinstance(x, np.ndarray) or isinstance(x, pd.DataFrame):
        return np.array(x)
    elif isinstance(x, np.float64):
        return np.array([x])
    elif isinstance(x, cudf.DataFrame) or isinstance(x, cudf.Series):
        return x.to_pandas().values
    return x    

# Run tests

In [ ]:
%%time
nrows = 2**16
ncols = 40

X = load_data(nrows, ncols)
print("Data shape:", X.shape)

In [ ]:
n_neighbors = 10

In [ ]:
%%time
knn_sk = skKNN(X)
D_sk, I_sk = knn_sk.query(X, n_neighbors)

In [ ]:
%%time
X = cudf.DataFrame.from_pandas(X)

In [ ]:
%%time
knn_cuml = cumlKNN(n_gpus=1)
knn_cuml.fit(X)
D_cuml, I_cuml = knn_cuml.query(X, n_neighbors)

In [ ]:
passed = array_equal(D_sk, D_cuml)
message = "compare knn: cuml vs sklearn distances %s" % ("equal" if passed else "NOT equal")
print(message)
passed = array_equal(I_sk, I_cuml)
message = "compare knn: cuml vs sklearn indexes %s" % ("equal" if passed else "NOT equal")
print(message)